In [1]:
import pickle
import random
from ase import Atoms, Atom
from ase.visualize import view
from ase.constraints import dict2constraint
from ase.calculators.singlepoint import SinglePointCalculator

In [2]:
data = pickle.load(open('Surface_Energy_data.pkl','rb'))
docs = data['docs']

In [3]:
docs[0]

{'mongo_id': ObjectId('5c0a78c1871ca7c005fcf28d'),
 'mpid': 'mp-580357',
 'formula': 'Cu6Pd2',
 'miller': [1, 0, 0],
 'intercept': 0.11824409146003978,
 'intercept_uncertainty': 0.009322162361717491,
 'initial_configuration': {'atoms': {'atoms': [{'symbol': 'Cu',
     'position': [1.82884281, 1.82884281, 13.716321075],
     'tag': 0,
     'index': 0,
     'charge': 0.0,
     'momentum': [0.0, 0.0, 0.0],
     'magmom': 0.0},
    {'symbol': 'Cu',
     'position': [1.82884281, 0.0, 11.887478265],
     'tag': 0,
     'index': 1,
     'charge': 0.0,
     'momentum': [0.0, 0.0, 0.0],
     'magmom': 0.0},
    {'symbol': 'Cu',
     'position': [0.0, 1.82884281, 11.887478265],
     'tag': 0,
     'index': 2,
     'charge': 0.0,
     'momentum': [0.0, 0.0, 0.0],
     'magmom': 0.0},
    {'symbol': 'Pd',
     'position': [0.0, 0.0, 13.716321075],
     'tag': 0,
     'index': 3,
     'charge': 0.0,
     'momentum': [0.0, 0.0, 0.0],
     'magmom': 0.0},
    {'symbol': 'Cu',
     'position': [1.8288

In [4]:
def make_atoms_from_doc(doc):
    '''
    This is the inversion function for `make_doc_from_atoms`; it takes
    Mongo documents created by that function and turns them back into
    an ase.Atoms object.

    Args:
        doc     Dictionary/json/Mongo document created by the
                `make_doc_from_atoms` function.
    Returns:
        atoms   ase.Atoms object with an ase.SinglePointCalculator attached
    '''
    atoms = Atoms([Atom(atom['symbol'],
                        atom['position'],
                        tag=atom['tag'],
                        momentum=atom['momentum'],
                        magmom=atom['magmom'],
                        charge=atom['charge'])
                   for atom in doc['atoms']['atoms']],
                  cell=doc['atoms']['cell'],
                  pbc=doc['atoms']['pbc'],
                  info=doc['atoms']['info'],
                  constraint=[dict2constraint(constraint_dict)
                              for constraint_dict in doc['atoms']['constraints']])
    results = doc['results']
    calc = SinglePointCalculator(energy=results.get('energy', None),
                                 forces=results.get('forces', None),
                                 stress=results.get('stress', None),
                                 atoms=atoms)
    atoms.set_calculator(calc)
    return atoms

In [6]:
initial_atoms = make_atoms_from_doc(docs[0]['initial_configuration'])
view(initial_atoms, viewer='x3d')